In [1]:
import os 
import torch
from ultralytics import YOLO
import shutil
import random

In [2]:
def split_dataset(images_dir, labels_dir, output_dir, test_ratio=0.8, seed=42):
    """
    Teilt einen Datensatz (Bilder + Labels) in Training und Validierung auf.

    Args:
        images_dir (str): Pfad zum Ordner mit den Bildern.
        labels_dir (str): Pfad zum Ordner mit den Labels.
        output_dir (str): Pfad zum Ordner, in dem die aufgeteilten Daten gespeichert werden sollen.
        test_ratio (float): Verhältnis der Testdaten (zwischen 0 und 1). Standard: 0.8.
        seed (int): Zufallssaat für Reproduzierbarkeit. Standard: 42.

    Returns:
        None
    """
    # Unterordner-Pfade
    train_images_dir = os.path.join(output_dir, "train/images")
    train_labels_dir = os.path.join(output_dir, "train/labels")
    val_images_dir = os.path.join(output_dir, "val/images")
    val_labels_dir = os.path.join(output_dir, "val/labels")
    
    # Vorhandene Ordner löschen, falls vorhanden
    if os.path.exists(os.path.join(output_dir, "train")):
        os.rmdir(os.path.join(output_dir, "train"))
    if os.path.exists(os.path.join(output_dir, "val")):
        os.rmdir(os.path.join(output_dir, "val"))
    
    # Ordner neu erstellen
    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    # Liste aller Bilder
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
    
    # Shuffle und Split
    random.seed(seed)
    random.shuffle(image_files)
    train_split = int(len(image_files) * (1 - test_ratio))

    train_files = image_files[:train_split]
    val_files = image_files[train_split:]

    # Dateien kopieren
    def copy_files(file_list, dest_images_dir, dest_labels_dir):
        for file in file_list:
            shutil.copy(os.path.join(images_dir, file), os.path.join(dest_images_dir, file))
            label_file = file.rsplit('.', 1)[0] + '.txt'
            if os.path.exists(os.path.join(labels_dir, label_file)):
                shutil.copy(os.path.join(labels_dir, label_file), os.path.join(dest_labels_dir, label_file))

    copy_files(train_files, train_images_dir, train_labels_dir)
    copy_files(val_files, val_images_dir, val_labels_dir)

    print(f"Train/Validation-Split abgeschlossen! Daten in '{output_dir}' gespeichert.")

# Beispielaufruf
BASE_DIR = os.getcwd()

yolo_images_dir = os.path.join(BASE_DIR, "Data", "Kugellager_Data", "YOLO_data", "yolo_images_dump")
yolo_labels_dir = os.path.join(BASE_DIR, "Data", "Kugellager_Data", "YOLO_data", "yolo_labels_dump")
yolo_output_dir = os.path.join(BASE_DIR, "Data", "Kugellager_Data", "YOLO_data")

split_dataset(images_dir=yolo_images_dir,
              labels_dir=yolo_labels_dir,
              output_dir=yolo_output_dir,
              test_ratio=0.2,
              seed=42)


Train/Validation-Split abgeschlossen! Daten in 'c:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_data' gespeichert.


In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0))
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

CUDA available: True
CUDA device name: NVIDIA GeForce RTX 3060 Ti
CUDA version: 12.4
cuDNN version: 90100


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Verwendung von yolv8m um eine Blance zwischen performance und Genauigkeit zu haben 
model = YOLO("yolov8m.pt")

# Train the model
path_to_yolo_yaml = os.path.join(BASE_DIR, "yaml_files", "yolo_dataset.yaml")
train_results = model.train(
    data=path_to_yolo_yaml,
    epochs=400,
    imgsz=150, # Weil das die tatsächliche Größe unserer Bilder darstellt
    device=device,
    batch=16,
    mosaic=1.0,
)

# Evaluate model performance on the validation set
metrics = model.val()
print(metrics)

# # Perform object detection on an image
# image_path = r"C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_Data\val\images\N (6155).png"
# results = model(image_path)
# results[0].show()

# Export the model to the same directory as the script
export_path = os.path.join(BASE_DIR, "Models", "yolo_kugellager_modell.pt")
model_path = model.export()
shutil.move(src=model_path, dst=export_path)
print(f"Model exported to: {export_path}")


Using device: cuda:0
Ultralytics 8.3.44  Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=c:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\yaml_files\yolo_dataset.yaml, epochs=400, time=None, patience=100, batch=16, imgsz=150, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embe

train: Scanning C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_Data\train\labels... 446 images, 49 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:00<00:00, 1029.40it/s]


train: New cache created: C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_Data\train\labels.cache


val: Scanning C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_Data\val\labels... 113 images, 10 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<00:00, 789.29it/s]

val: New cache created: C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_Data\val\labels.cache


...


313 epochs completed in 0.281 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 52.0MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics 8.3.44  Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.97it/s]


                   all        120        148      0.778      0.759      0.817      0.634
                defect         57         83      0.797      0.747      0.825       0.51
             no defect         29         29      0.835      0.875      0.906      0.842
         contamination         28         36      0.702      0.654       0.72      0.551
Speed: 0.0ms preprocess, 1.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train9
Ultralytics 8.3.44  Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_Data\val\labels.cache... 113 images, 10 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.75it/s]


                   all        120        148       0.78      0.767      0.816       0.63
                defect         57         83      0.797      0.747      0.825      0.507
             no defect         29         29      0.837      0.887        0.9      0.836
         contamination         28         36      0.706      0.667      0.722      0.547
Speed: 0.1ms preprocess, 4.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train92
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026E89AD88D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.01201

In [5]:
metrics = model.val()
# print(f"Precision: {metrics['precision']}")
# print(f"Recall: {metrics['recall']}")
# print(f"mAP@0.5: {metrics['map50']}")
# print(f"mAP@0.5:0.95: {metrics['map']}")

Ultralytics 8.3.44  Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)


val: Scanning C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\Data\Kugellager_Data\YOLO_Data\val\labels.cache... 113 images, 10 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.74it/s]


                   all        120        148       0.78      0.767      0.816       0.63
                defect         57         83      0.797      0.747      0.825      0.507
             no defect         29         29      0.837      0.887        0.9      0.836
         contamination         28         36      0.706      0.667      0.722      0.547
Speed: 0.1ms preprocess, 4.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train93
